# Random Forest Regression from Scratch

Group 18 Members:

- Clara Pichler, 11917694
- Hannah Knapp, 11901857 
- Sibel Toprakkiran, 09426341

### Overview

1. Bootstraping
- `make_bootstraps(df, n_bootstraps=100)`

2. Decision Tree Regression
- `mse(y)`
- `split_dataset(X, y, feature_idx, threshold)`
- `find_best_split(X, y)`
- `build_tree(X, y, max_depth, min_samples_split, depth=0)`
- `predict_tree(tree, X)`

3. Random Forest Regression

4. Random Forest Regression - LLM
- 

5. Evaluation
- Ours
- LLM
- sklearn

In [250]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, confusion_matrix

We will use the data set `mountains_vs_beaches_preferences.csv` as a data frame for testing our functions.

In [251]:
df_pref = pd.read_csv('data/mountains_vs_beaches_preferences.csv')

df_pref_one_hot = pd.get_dummies(df_pref, columns=['Location', 'Favorite_Season', 'Gender'], drop_first=True)

target_mean = df_pref_one_hot.groupby('Preferred_Activities')['Preference'].mean()
df_pref_one_hot['Preferred_Activities_Encoded'] = df_pref_one_hot['Preferred_Activities'].map(target_mean)

education_mapping = {'high school': 0, 'bachelor': 1, 'master': 2, 'doctorate': 3}
df_pref_one_hot['Education_Level_Encoded'] = df_pref_one_hot['Education_Level'].map(education_mapping)

df_pref_one_hot = df_pref_one_hot.drop(["Education_Level", "Preferred_Activities"], axis=1)

display(df_pref_one_hot.info())
display(df_pref_one_hot.isnull().sum())
display(df_pref_one_hot.head())

display(df_pref_one_hot['Preference'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52444 entries, 0 to 52443
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age                           52444 non-null  int64  
 1   Income                        52444 non-null  int64  
 2   Travel_Frequency              52444 non-null  int64  
 3   Vacation_Budget               52444 non-null  int64  
 4   Proximity_to_Mountains        52444 non-null  int64  
 5   Proximity_to_Beaches          52444 non-null  int64  
 6   Pets                          52444 non-null  int64  
 7   Environmental_Concerns        52444 non-null  int64  
 8   Preference                    52444 non-null  int64  
 9   Location_suburban             52444 non-null  bool   
 10  Location_urban                52444 non-null  bool   
 11  Favorite_Season_spring        52444 non-null  bool   
 12  Favorite_Season_summer        52444 non-null  bool   
 13  F

None

Age                             0
Income                          0
Travel_Frequency                0
Vacation_Budget                 0
Proximity_to_Mountains          0
Proximity_to_Beaches            0
Pets                            0
Environmental_Concerns          0
Preference                      0
Location_suburban               0
Location_urban                  0
Favorite_Season_spring          0
Favorite_Season_summer          0
Favorite_Season_winter          0
Gender_male                     0
Gender_non-binary               0
Preferred_Activities_Encoded    0
Education_Level_Encoded         0
dtype: int64

,Age,Income,Travel_Frequency,Vacation_Budget,Proximity_to_Mountains,Proximity_to_Beaches,Pets,Environmental_Concerns,Preference,Location_suburban,Location_urban,Favorite_Season_spring,Favorite_Season_summer,Favorite_Season_winter,Gender_male,Gender_non-binary,Preferred_Activities_Encoded,Education_Level_Encoded
0,56,71477,9,2477,175,267,0,1,1,False,True,False,True,False,True,False,0.500114,1
1,69,88740,1,4777,228,190,0,1,0,True,False,False,False,False,True,False,0.000000,2
2,46,46562,0,1469,71,280,0,0,1,False,True,False,False,True,False,False,0.500114,2
3,32,99044,6,1482,31,255,1,0,1,False,False,False,True,False,False,True,0.500916,0
4,60,106583,5,516,23,151,1,1,0,True,False,False,False,True,False,False,0.000000,0


Preference
0    39296
1    13148
Name: count, dtype: int64

In [252]:
df_airfol = pd.read_csv("./data/airfoil_noise_data.csv")

display(df_airfol.info())
display(df_airfol.isnull().sum())
display(df_airfol.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503 entries, 0 to 1502
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x0      1503 non-null   int64  
 1   x1      1503 non-null   float64
 2   x2      1503 non-null   float64
 3   x3      1503 non-null   float64
 4   x4      1503 non-null   float64
 5   y       1503 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 70.6 KB


None

x0    0
x1    0
x2    0
x3    0
x4    0
y     0
dtype: int64

,x0,x1,x2,x3,x4,y
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


## Bootstrapping

Bootstrapping is a method to create multiple subsets of the original data by sampling with replacement. Each subset is used to train one decision tree in the forest.

- Introduces randomness, ensuring that trees see different views of the data.
- Helps in reducing overfitting by decorrelating the trees.

- __Sample Size__: The size of the bootstrap sample is the same as the original dataset.
- __Replacement__: Sampling with replacement ensures diversity between bootstrapped samples.


In [253]:
def make_bootstraps(df, n_bootstraps=100):
    
    dic_boot = {}
    sample_size = df.shape[0]
    idx = [i for i in range(sample_size)]

    for b in range(n_bootstraps):
        
        sidx   = np.random.choice(idx,replace=True,size=sample_size)
        b_samp = df.iloc[sidx,:]
        
        dic_boot['boot_'+str(b)] = {'boot':b_samp}
    
    return(dic_boot)

How our function `make_bootstraps` works:
1. Each data point has equal probability of being selected 
2. Selecting data points from the original sample for the current bootstrap sample, with replacement! Until we reached the same size as the original data
4. Repeating this process until we have `n_bootstraps` bootstrap samples which we save in a dictonary `dic_boot`

In [254]:
dic_boot_pref = make_bootstraps(df_pref_one_hot)

In [255]:
display(dic_boot_pref['boot_0']['boot'].head(5))
display(dic_boot_pref['boot_0']['boot'].info())

,Age,Income,Travel_Frequency,Vacation_Budget,Proximity_to_Mountains,Proximity_to_Beaches,Pets,Environmental_Concerns,Preference,Location_suburban,Location_urban,Favorite_Season_spring,Favorite_Season_summer,Favorite_Season_winter,Gender_male,Gender_non-binary,Preferred_Activities_Encoded,Education_Level_Encoded
19936,32,95402,4,3399,18,64,1,1,0,False,False,False,True,False,True,False,0.000000,1
10250,33,62734,5,4104,295,31,1,0,0,False,True,True,False,False,False,False,0.500916,0
35515,22,46106,8,4022,255,0,1,1,0,True,False,True,False,False,False,False,0.500916,2
50737,38,110578,0,4791,299,288,0,0,0,True,False,False,True,False,True,False,0.000000,3
50138,68,64011,6,872,207,263,1,1,1,False,True,False,False,True,False,True,0.500916,3


<class 'pandas.core.frame.DataFrame'>
Index: 52444 entries, 19936 to 16920
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age                           52444 non-null  int64  
 1   Income                        52444 non-null  int64  
 2   Travel_Frequency              52444 non-null  int64  
 3   Vacation_Budget               52444 non-null  int64  
 4   Proximity_to_Mountains        52444 non-null  int64  
 5   Proximity_to_Beaches          52444 non-null  int64  
 6   Pets                          52444 non-null  int64  
 7   Environmental_Concerns        52444 non-null  int64  
 8   Preference                    52444 non-null  int64  
 9   Location_suburban             52444 non-null  bool   
 10  Location_urban                52444 non-null  bool   
 11  Favorite_Season_spring        52444 non-null  bool   
 12  Favorite_Season_summer        52444 non-null  bool   
 13  Fa

None

In [256]:
duplicates_boot_0 = dic_boot_pref['boot_0']['boot'].duplicated()
display(duplicates_boot_0.sum())
display(dic_boot_pref['boot_1']['boot'].duplicated().sum())
display(dic_boot_pref['boot_2']['boot'].duplicated().sum())

19288

19386

19334

## Decision Tree Regression

First, we define the functions to split the data and calculate metrics like the mean squared error (MSE).
And for the actual building of the trees we recursively split the data into smaller groups, based on feature thresholds, until a stopping condition is met (e.g., max depth or minimum samples per leaf).


In [257]:
def mse(y):
    return np.mean((y - np.mean(y))**2)

In [258]:
def mse_split(y_left, y_right):

    total = len(y_left) + len(y_right)
    weighted_mse = (len(y_left) * mse(y_left) + len(y_right) * mse(y_right)) / total

    return weighted_mse


def variance_reduction_split(y_left, y_right):

    total = len(y_left) + len(y_right)
    parent_variance = np.var(np.concatenate([y_left, y_right]))
    weighted_variance = (len(y_left) * np.var(y_left) + len(y_right) * np.var(y_right)) / total

    return parent_variance - weighted_variance


def mae_split(y_left, y_right):

    total = len(y_left) + len(y_right)
    weighted_mae = (len(y_left) * np.mean(np.abs(y_left - np.mean(y_left))) +
                    len(y_right) * np.mean(np.abs(y_right - np.mean(y_right)))) / total
    
    return weighted_mae



def entropy(y):

    if len(y) == 0:
        return 0
    
    value_counts = np.unique(y, return_counts=True)[1]
    probabilities = value_counts / len(y)
    
    return -np.sum(probabilities * np.log2(probabilities))


def information_gain(y_parent, y_left, y_right):

    n_total = len(y_parent)
    n_left = len(y_left)
    n_right = len(y_right)
    
    if n_left == 0 or n_right == 0:
        return 0  # No information gain if one side is empty
    
    parent_entropy = entropy(y_parent)
    weighted_child_entropy = (n_left / n_total) * entropy(y_left) + (n_right / n_total) * entropy(y_right)
    
    return parent_entropy - weighted_child_entropy


def information_gain_split(y_left, y_right):
    
    return -information_gain(np.concatenate([y_left, y_right]), y_left, y_right)




`mse(y)` calculates how spread out the data points are from their mean. A lower MSE indicates that the values in `y` are closer to the mean, which means the split effectively reduces variability. 

**Mean Squared Error (MSE)**

- **Definition**: Measures the average squared difference between the predicted and actual values.
- **Usage**: Best suited for regression problems where the target is continuous.
- **Pros**: Penalizes larger errors more than smaller ones, making it sensitive to outliers.
- **Cons**: Overly sensitive to outliers.



**Variance Reduction**

- **Definition**: Measures how much the variance of the target variable decreases when the data is split.
- **Formula**:
  \[
  \text{Variance Reduction} = \text{Variance (parent)} - \text{Weighted Variance (left and right children)}
  \]
  Variance is calculated as:
  \[
  \text{Variance} = \frac{1}{N} \sum_{i=1}^{N} (y_i - \bar{y})^2
  \]
- **Usage**: Another metric for regression, similar to MSE but focuses on reducing variance.
- **Pros**: Intuitive and directly measures how well a split homogenizes the data.
- **Cons**: Can be slower to compute compared to simpler metrics.



**Reduction in Standard Deviation (SD Reduction)**
- **Definition**: Measures the reduction in standard deviation after the split.
- **Formula**:
  \[
  \text{SD Reduction} = \text{Standard Deviation (parent)} - \text{Weighted SD (left and right children)}
  \]
- **Usage**: Also for regression; directly interprets spread in the target.
- **Pros**: Similar to variance reduction but easier to interpret since it's in the same units as the target.
- **Cons**: Like variance reduction, it may still suffer if the data has extreme outliers.



**Information Gain**

- **Definition**: Measures the reduction in entropy (uncertainty) of the target after a split.
- **Formula**:
  \[
  \text{Information Gain} = \text{Entropy (parent)} - \text{Weighted Entropy (left and right children)}
  \]
  Where entropy is:
  \[
  \text{Entropy} = - \sum_{i} p_i \log_2(p_i)
  \]
  Here, \( p_i \) is the proportion of each class.
- **Usage**: Often used for classification, but can be adapted for regression using a continuous version of entropy.
- **Pros**: Captures the "uncertainty reduction" of a split.
- **Cons**: Computationally intensive for large datasets or many splits.



**Which Metric Should You Use?**

   - **MSE** or **Variance Reduction**: Best for datasets without significant outliers.
   - **MAE**: Better for datasets with outliers or non-symmetric error distributions.
   - **SD Reduction**: Useful if you want an intuitive measure.



In [259]:
def split_dataset(X, y, feature_idx, threshold):
    
    left_mask = X[:, feature_idx] <= threshold
    right_mask = ~left_mask

    return X[left_mask], X[right_mask], y[left_mask], y[right_mask]

`split_dataset` divides X (features) and y (target) into two groups based on whether the value of a feature is less than or equal to a given threshold. This is used to evaluate potential splits during training. For example split on feature `Unemployment` at value 6 creates two groups: rows where `Unemployment <= 6` and rows where `Unemployment > 6`.

In [260]:
def find_best_split(X, y, feature_subset=None, metric="mse"):
    print('find best split')
    best_feature, best_threshold = None, None
    best_metric_value = -float("inf")
    features = feature_subset if feature_subset is not None else range(X.shape[1])

    metric_functions = {
        "mse": mse_split,
        "variance_reduction": variance_reduction_split,
        "mae": mae_split,
        "information_gain": information_gain_split,
    }
    
    metric_function = metric_functions[metric]
    
    for feature_idx in features:
        thresholds = np.unique(X[:, feature_idx])
        print('thresholds')
        print(thresholds)
        for threshold in thresholds:
            _, _, y_left, y_right = split_dataset(X, y, feature_idx, threshold)
            
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            
            metric_value = metric_function(y_left, y_right)
            print('metric_value =' + str(metric_value))
            print('best_metric_value =' + str(best_metric_value))
            if metric_value > best_metric_value:
                best_metric_value = metric_value
                best_feature = feature_idx
                best_threshold = threshold
    
    return best_feature, best_threshold


`find_best_split` evaluates all possible splits for every feature and every threshold. It selects the split according to the metric that was given to it.

The output is the `best_feature` (column index of the splitting feature) and the `best_threshold` (value of the feature where the split happens).

In [261]:
def build_tree(X, y, max_depth, min_samples_split, depth=0, metric="mse"):
    print('max_depth ' + str(max_depth))
    print('min_samples_split ' + str(min_samples_split))
    if depth >= max_depth or len(y) < min_samples_split: # or mse(y) == 0:
        print('thats a node')
        return np.mean(y)  

    n_features = X.shape[1]
    #feature_subset = np.random.choice(n_features, size=int(np.sqrt(n_features)), replace=False)
    feature_subset = np.random.choice(n_features, size=int(n_features), replace=False)
    print(feature_subset)
    feature_idx, threshold = find_best_split(X, y, feature_subset, metric=metric)
    
    if feature_idx is None:
        return np.mean(y)  
    
    X_left, X_right, y_left, y_right = split_dataset(X, y, feature_idx, threshold)
    print('X-left')
    print(X_left)
    print('Y-left')
    print(X_right)
    left_subtree = build_tree(X_left, y_left, max_depth, min_samples_split, depth + 1, metric)
    right_subtree = build_tree(X_right, y_right, max_depth, min_samples_split, depth + 1, metric)
    
    return {
        "feature_idx": feature_idx,
        "threshold": threshold,
        "left": left_subtree,
        "right": right_subtree,
    }

- __Stopping Conditions__: Stops if the max depth is reached, if there are fewer samples than min_samples_split, or if the MSE is 0 (all values are the same).
- __Recursive Splitting__: For each split, the function creates a left and right subtree until the stopping conditions are met.
- __Leaf Node__: If the recursion stops, the tree stores the mean value of y for prediction.
- __Feature Selection__: Chooses $\sqrt{n}$ features randomly for each tree, where $n$ is the number of original features

In [262]:
def predict_tree(tree, X):

    if isinstance(tree, dict):

        feature_idx = tree["feature_idx"]
        threshold = tree["threshold"]

        if X[feature_idx] <= threshold:
            return predict_tree(tree["left"], X)
        
        else:
            return predict_tree(tree["right"], X)
        
    else:
        return tree  # Leaf node


Traverses the tree based on the input features until you reach a leaf node.
Returns the mean value of the target variable y at the leaf node.

## Random Forest Regression

In [263]:
class RandomForestRegressor_18:
    
    def __init__(self, n_trees=10, max_depth=5, min_samples_split=10, metric="mse"):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []
        self.metric = metric
        
    def fit(self, X, y):

        bootstraps = make_bootstraps(pd.DataFrame(np.hstack((X, y[:, None]))), n_bootstraps=self.n_trees)

        for b in range(self.n_trees):
            print('tree ' + str(b))
            print('----------------')
            bootstrap = bootstraps[f'boot_{b}']['boot']
            X_boot = bootstrap.iloc[:, :-1].values
            y_boot = bootstrap.iloc[:, -1].values
            print(X_boot)
            print(y_boot)
            tree = build_tree(X_boot, y_boot, self.max_depth, self.min_samples_split, metric=self.metric)
            self.trees.append(tree)
    
    def predict(self, X):
        predictions = np.array([predict_tree(tree, x) for x in X for tree in self.trees])
        predictions = predictions.reshape(self.n_trees, len(X))

        return np.mean(predictions, axis=0)

## Evaluation

We use for the comparison of the methods the same holdout split with the training set containg 75% of the data.

In [264]:
# X = df_pref_one_hot.drop("Preference", axis=1).values
# y = df_pref_one_hot["Preference"].values
df_airfol = df_airfol.head(5)
X = df_airfol.drop("y", axis=1).values
y = df_airfol["y"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

For evaluation we use the following three methods:

__Mean Squared Error (MSE)__

The average of the squared differences between the predicted values and the actual values. It gives more weight to larger errors.

- A smaller MSE value indicates that the model’s predictions are close to the actual values.
- Since it's based on squared differences, large prediction errors (outliers) have a greater impact.
- MSE is in the square of the unit of your target variable
- Useful when large errors are particularly undesirable and need to be penalized more heavily.



__Mean Absolute Error (MAE)__

The average of the absolute differences between predicted values and actual values. Unlike MSE, it treats all errors equally, regardless of size.

- A smaller MAE value indicates better model performance.
- MAE is in the same unit as the target variable, making it more interpretable compared to MSE.
- Good for understanding the typical size of prediction errors.
- Less sensitive to outliers compared to MSE.

__R-squared__

The proportion of variance in the target variable that the model explains. It ranges from:
- __1__: Perfect fit (model explains all variance in the data).
- __0__: Model does no better than predicting the mean of the target.
- __Negative__: Model performs worse than simply predicting the mean.


- A higher R-squared (close to 1) indicates a good fit.
- A low or negative R-squared suggests that your model is not capturing the relationship between the features and target effectively.
- Helps understand how well the model explains the variability in the target variable.
- Not ideal for measuring absolute error but useful for comparing models.

### Our Regressor

In [265]:
rf_18 = RandomForestRegressor_18(n_trees=10, max_depth=5, min_samples_split=1, metric="variance_reduction")
rf_18.fit(X_train, y_train)

predictions_18 = rf_18.predict(X_test)
predictions_18

tree 0
----------------
[[1.60000e+03 0.00000e+00 3.04800e-01 7.13000e+01 2.66337e-03]
 [8.00000e+02 0.00000e+00 3.04800e-01 7.13000e+01 2.66337e-03]
 [1.25000e+03 0.00000e+00 3.04800e-01 7.13000e+01 2.66337e-03]]
[127.591 126.201 125.951]
max_depth 5
min_samples_split 1
[0 1 3 4 2]
find best split
thresholds
[ 800. 1250. 1600.]
metric_value =0.0722000000000001
best_metric_value =-inf
metric_value =0.5100500000000004
best_metric_value =0.0722000000000001
thresholds
[0.]
thresholds
[71.3]
thresholds
[0.00266337]
thresholds
[0.3048]
X-left
[[8.00000e+02 0.00000e+00 3.04800e-01 7.13000e+01 2.66337e-03]
 [1.25000e+03 0.00000e+00 3.04800e-01 7.13000e+01 2.66337e-03]]
Y-left
[[1.60000e+03 0.00000e+00 3.04800e-01 7.13000e+01 2.66337e-03]]
max_depth 5
min_samples_split 1
[1 0 2 3 4]
find best split
thresholds
[0.]
thresholds
[ 800. 1250.]
metric_value =0.015625
best_metric_value =-inf
thresholds
[0.3048]
thresholds
[71.3]
thresholds
[0.00266337]
X-left
[[8.00000e+02 0.00000e+00 3.04800e-01 7.1

array([126.607, 127.263])

In [266]:
mse_18 = mean_squared_error(y_test, predictions_18)
mae_18 = mean_absolute_error(y_test, predictions_18)
r2_18 = r2_score(y_test, predictions_18)

print("Model Performance:")
print(f"Mean Squared Error: {mse_18:.2f}")
print(f"Mean Absolute Error: {mae_18:.2f}")
print(f"R-squared: {r2_18:.2f}")


Model Performance:
Mean Squared Error: 1.01
Mean Absolute Error: 0.80
R-squared: 0.21


### SkLearn Regressor

In [235]:
from sklearn.ensemble import RandomForestRegressor

In [236]:
rf_sklearn = RandomForestRegressor(n_estimators=10, max_depth = 5, min_samples_split = 2)
rf_sklearn.fit(X_train, y_train)

predictions_sklearn = rf_sklearn.predict(X_test)

In [237]:
mse_sklearn = mean_squared_error(y_test, predictions_sklearn)
mae_sklearn = mean_absolute_error(y_test, predictions_sklearn)
r2_sklearn = r2_score(y_test, predictions_sklearn)

print("Model Performance:")
print(f"Mean Squared Error: {mse_sklearn:.2f}")
print(f"Mean Absolute Error: {mae_sklearn:.2f}")
print(f"R-squared: {r2_sklearn:.2f}")

Model Performance:
Mean Squared Error: 0.45
Mean Absolute Error: 0.49
R-squared: 0.65


## Notes

Max Depth: Prevents trees from growing too deep, which could lead to overfitting.

Min Samples Split: Controls the smallest group size allowed for further splitting, preventing unnecessary splits.


**Key Insights from These Metrics**
1. **MSE**:
   - If it's high, your model is making some large errors that need to be addressed.
   - If it's low, your model is capturing most of the relationship.

2. **MAE**:
   - Directly tells you the average prediction error. 
   - Compare it to the scale of your target variable; if MAE is relatively low, the model is performing well.

3. **R²**:
   - A high R² suggests the model explains a significant portion of the target variable's variance.
   - If R² is low (or negative), consider if the features are truly predictive of the target or if the model is too simple/complex.


**Next Steps Based on Metrics**

- **High MSE or MAE**:
  - Investigate outliers, or whether the model needs better hyperparameter tuning.
  - Consider adding more predictive features or improving feature engineering.

- **Low R²**:
  - Evaluate if features are relevant or add more features to capture the variance.
  - Consider if the model is underfitting or overfitting:
    - Underfitting: Increase `max_depth`, add more `n_trees`.
    - Overfitting: Decrease `max_depth` or regularize.
